## Word2Vec

### (1) 데이터 불러오기

In [90]:
from gensim.models import Word2Vec
from konlpy.tag import Okt
from konlpy.tag import Kkma
import time
import os
import numpy as np
import pandas as pd

In [10]:
data = pd.read_csv('./data/clean_review_dataset.csv')
data = data.drop('Unnamed: 0', axis=1)
data

,영화,리뷰,평점,site
0,명량,여기에 말같잖은 평다는 놈들은 매국노 인증인 듯,10.0,naver
1,명량,다 좋았어요 한산보고 나서 보니 더 좋네요,10.0,naver
2,명량,2번째 본 역사영화 진짜 이순신은 대단한 영웅 중에 영웅보는 내내 끝까지 긴장 놓을...,10.0,naver
3,명량,정말 거짓말 안 한다클레멘타인에 대한 평가를 호평으로 오해했으면 관객 수가 이리 나...,1.0,naver
4,명량,재미있게 잘 봤습니다,10.0,naver
...,...,...,...,...
106885,완득이,세상에 수호천사는 없다,2.0,watcha
106886,완득이,도대체 유아인은 영화를 찍는지,1.0,watcha
106887,완득이,나도 멘토가 있었다면,6.0,watcha
106888,완득이,가장 이상적인 한국형 성장드라마,7.0,watcha


In [103]:
len(data['리뷰'].unique())

103574

In [104]:
data.drop_duplicates(subset=['리뷰'], inplace=True) # reviews 열에서 중복인 내용이 있다면 중복 제거
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 103574


In [110]:
data.to_csv('./data/clean_review_dataset.csv', index=)

,영화,리뷰,평점,site
0,명량,여기에 말같잖은 평다는 놈들은 매국노 인증인 듯,10.0,naver
1,명량,다 좋았어요 한산보고 나서 보니 더 좋네요,10.0,naver
2,명량,2번째 본 역사영화 진짜 이순신은 대단한 영웅 중에 영웅보는 내내 끝까지 긴장 놓을...,10.0,naver
3,명량,정말 거짓말 안 한다클레멘타인에 대한 평가를 호평으로 오해했으면 관객 수가 이리 나...,1.0,naver
4,명량,재미있게 잘 봤습니다,10.0,naver
...,...,...,...,...
106885,완득이,세상에 수호천사는 없다,2.0,watcha
106886,완득이,도대체 유아인은 영화를 찍는지,1.0,watcha
106887,완득이,나도 멘토가 있었다면,6.0,watcha
106888,완득이,가장 이상적인 한국형 성장드라마,7.0,watcha


### (2) Okt로 명사 단어 추출

In [63]:
okt = Okt() 

reviews = data['리뷰']

texts = []
for review in reviews:
    words = []
    for word, pos in okt.pos(review):
        if pos in ['Adjective', 'Verb', 'Noun']:
            words.append(word)
            
    text = ' '.join(words)
    texts.append(words)

In [29]:
# 측정 시작
start = time.time()
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('1) 형태소에서 명사만 추출 시작')
okt = Okt()
okt_nouns = [okt.nouns(sentence) for sentence in data['리뷰']]
print('2) 형태소에서 명사만 추출 완료', time.time() - start)

1) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료 174.40225839614868


In [49]:
okt_nouns

[['여기', '말', '평', '놈', '매국노', '인증', '듯'],
 ['한산', '보고', '더'],
 ['역사영화',
  '진짜',
  '이순신',
  '영웅',
  '중',
  '영웅',
  '내내',
  '끝',
  '긴장',
  '수',
  '이순신',
  '인물',
  '정말',
  '영웅',
  '임'],
 ['정말', '거짓말', '안', '클레멘타인', '대한', '평가', '호평', '오해', '관객', '수가', '수도', '생각'],
 [],
 ['선용', '감'],
 ['그냥', '지루', '자체'],
 ['사람', '영화', '대한', '평점', '이순신', '대한', '평점', '국뽕', '눈', '것', '좀', '자제'],
 ['루지', '마', '이름', '일본', '장수', '거', '점줌'],
 ['척', '척', '대결', '아주', '해상전쟁', '영화'],
 ['이순신', '모독', '영화', '대가리', '이순신', '드', '보이', '찍'],
 ['이순신', '영화'],
 [],
 ['충족'],
 ['명량해전', '하나'],
 ['개국', '뽕', '영화', '덕', '끄덕', '수준'],
 ['굿'],
 ['그냥', '내내', '함', '나라'],
 ['말모', '대한민국', '최고'],
 ['최민식', '연기', '정말'],
 ['가슴', '대작'],
 ['두', '시간', '시간', '줄', '모름'],
 ['나침반',
  '등대',
  '첨단',
  '기술',
  '옛날',
  '자연',
  '지형',
  '이용',
  '저런',
  '전술',
  '질',
  '왜놈',
  '틈',
  '바구니',
  '나라',
  '구한',
  '역사',
  '국뽕',
  '뇨',
  '대해',
  '마음',
  '수'],
 ['진짜', '굿재밋', '엇어', '감동', '최', '고스'],
 ['진짜', '디테', '감상', '평', '가치'],
 ['최근',
  '극장',
  '한산',
  '보고',
  '티빙

### (3) Kkma로 명사 단어 추출

In [57]:
# 측정 시작
start = time.time()
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('형태소에서 명사만 추출 시작')
kkma = Kkma()
kkma_nouns = [kkma.nouns(sentence) for sentence in data['리뷰']]
print('형태소에서 명사만 추출 완료', time.time() - start)

형태소에서 명사만 추출 시작


java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: Java heap space

### (4) Word2Vec 모델

#### 1) 학습

In [83]:
data['리뷰'].unique()

array(['여기에 말같잖은 평다는 놈들은 매국노 인증인 듯', '다 좋았어요 한산보고 나서 보니 더 좋네요',
       '2번째 본 역사영화 진짜 이순신은 대단한 영웅 중에 영웅보는 내내 끝까지 긴장 놓을 수 없고 이순신이란 인물이 정말 대단한 영웅임',
       ..., '나도 멘토가 있었다면', '가장 이상적인 한국형 성장드라마', '멋진 조화사회 문제를 영화에 담아내긴 했다'],
      dtype=object)

In [66]:
start = time.time()
# word2vec 모델 학습
print('word2vec 모델 학습 시작')
model = Word2Vec(sentences=texts, vector_size=200, window=4, min_count=1, sg=1)
# skip-gram, window size=4, 벡터 크기 : 200, 최소 2번 이상 발생 단어만 학습
print('word2vec 모델 학습 완료: ', time.time() - start)

word2vec 모델 학습 시작
word2vec 모델 학습 완료:  21.269087076187134


In [71]:
model.wv.most_similar('영화')

[('화영', 0.8448615670204163),
 ('역사영화', 0.7615429162979126),
 ('화기', 0.758078932762146),
 ('대작', 0.754793643951416),
 ('공포영화', 0.7490036487579346),
 ('화란', 0.7481001615524292),
 ('만화영화', 0.7445890307426453),
 ('인건', 0.7434040307998657),
 ('명화', 0.7412079572677612),
 ('화이자', 0.7399641275405884)]

In [33]:
start = time.time()
# word2vec 모델 학습
print('word2vec 모델 학습 시작')
model = Word2Vec(sentences=okt_nouns, vector_size=200, window=4, min_count=2, sg=1)
# skip-gram, window size=4, 벡터 크기 : 200, 최소 2번 이상 발생 단어만 학습
print('word2vec 모델 학습 완료: ', time.time() - start)

word2vec 모델 학습 시작
word2vec 모델 학습 완료:  7.731443643569946


In [54]:
model.wv.vectors.shape

(15815, 200)

#### 2) 저장

In [34]:
start = time.time()
# 모델 저장
print('학습된 모델 저장 시작')
model.save('./data/okt_nouns.model')     
print('학습된 모델 저장 완료: ', time.time() - start)

학습된 모델 저장 시작
학습된 모델 저장 완료:  0.49311113357543945


#### 3) 모델 load

In [36]:
from gensim.models import Word2Vec
# 모델 로딩
model = Word2Vec.load('./data/okt_nouns.model')
model.corpus_total_words

735777

### (4) 단어 유사도 (코사인 유사도)

In [43]:
# 단어 유사도 계산 (코사인 유사도 기준)
model.wv.similarity(w1='영화', w2='어벤저스')

0.47109693

In [45]:
model.wv.similarity(w1='배우', w2='내공')

0.7428517

In [44]:
model.wv.most_similar('배우')

[('연기자', 0.8664340376853943),
 ('조연', 0.7954868078231812),
 ('배역', 0.7825795412063599),
 ('주연', 0.748223602771759),
 ('명품', 0.7479346990585327),
 ('호흡', 0.744335412979126),
 ('내공', 0.7428516745567322),
 ('신인', 0.7415962219238281),
 ('한몫', 0.7384046316146851),
 ('궁합', 0.7381974458694458)]